In [0]:
# Required each time the cluster is restarted which should be only on the first notebook as they run in order 

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@obitosalukedbxpj.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
 
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
# Costructing API URL with start and end date formated for geojson output

url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    response = requests.get(url)
    response.raise_for_status() # Raise HTTPError for bad responses
    data = response.json().get('features', [])

    if not data:
        print("No earthquake data found for the specified date range.")
    else:
        file_path = f"{bronze_adls}{start_date}_earthquake_data.json"

        json_data = json.dumps(data)
        dbutils.fs.put(file_path, json_data, overwrite=True)

        print(f"Earthquake data saved to {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data from the API: {e}")


Wrote 191016 bytes.
Earthquake data saved to abfss://bronze@obitosalukedbxpj.dfs.core.windows.net/2025-05-13_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 1.6,
  'place': '2 km SSW of La Mirada, CA',
  'time': 1747180756550,
  'updated': 1747204054704,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41152424',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci41152424&format=geojson',
  'felt': 1,
  'cdi': 1,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 39,
  'net': 'ci',
  'code': '41152424',
  'ids': ',ci41152424,',
  'sources': ',ci,',
  'types': ',dyfi,nearby-cities,origin,phase-data,scitech-link,',
  'nst': 22,
  'dmin': 0.0304,
  'rms': 0.25,
  'gap': 123,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - 2 km SSW of La Mirada, CA'},
 'geometry': {'type': 'Point', 'coordinates': [-118.022, 33.9003333, 12.01]},
 'id': 'ci41152424'}

In [0]:
# Exporting ADLS links and variables for use in other notebooks

output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

dbutils.jobs.taskValues.set(key= "bronze_output", value = output_data)